<a href="https://colab.research.google.com/github/gtbnhyujmj/-Good-Auto_Shorts_Maker/blob/main/%5BV2%5D_Stickered_Movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os  # 匯入標準/外部函式庫
import cv2  # 匯入標準/外部函式庫
import numpy as np  # 匯入標準/外部函式庫
from PIL import Image  # 從套件匯入模組或類別
import random  # 匯入標準/外部函式庫

# def Area

In [4]:
# ===== 函數：合成透明圖層（快速 numpy） =====
def blend_with_alpha_fast(base_img, overlay_imgs):  # 定義函數：blend_with_alpha_fast
    base_rgba = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGBA)  # OpenCV 影像處理函式
    for overlay in overlay_imgs:  # for 迴圈
        overlay_rgba = np.array(overlay.convert('RGBA').resize((base_rgba.shape[1], base_rgba.shape[0]), Image.LANCZOS))  # numpy 相關函式
        alpha_overlay = overlay_rgba[..., 3:] / 255.0  # 變數指派
        alpha_base = base_rgba[..., 3:] / 255.0  # 變數指派
        out_alpha = alpha_overlay + alpha_base * (1 - alpha_overlay)  # 變數指派
        base_rgba[..., :3] = (overlay_rgba[..., :3] * alpha_overlay + base_rgba[..., :3] * alpha_base * (1 - alpha_overlay)) / (out_alpha + 1e-6)  # 變數指派
        base_rgba[..., 3:] = out_alpha * 255  # 變數指派
    result_bgr = cv2.cvtColor(base_rgba, cv2.COLOR_RGBA2BGR)  # OpenCV 影像處理函式
    return result_bgr  # 回傳結果

In [5]:
# ===== 函數：處理影片，每幀合成 PNG 疊圖 =====
def overlay_multiple_pngs_on_video_fast(video_path, overlay_paths, output_path):  # 定義函數：overlay_multiple_pngs_on_video_fast
    cap = cv2.VideoCapture(video_path)  # OpenCV 影像處理函式
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # OpenCV 影像處理函式
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # OpenCV 影像處理函式
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # OpenCV 影像處理函式
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # OpenCV 影像處理函式
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # OpenCV 影像處理函式
    overlay_images = [Image.open(path).resize((width, height), Image.LANCZOS) for path in overlay_paths]  # PIL 影像處理
    while True:
        ret, frame = cap.read()  # 變數指派
        if not ret:  # if 判斷式
            break
        result_frame = blend_with_alpha_fast(frame, overlay_images)  # 變數指派
        out.write(result_frame)
    cap.release()
    out.release()
    print(f"✅ 影片已產出：{output_path}")

# Main Precessing

In [6]:
# ===== 資料夾設定 =====
BACKGROUND_DIR = "/content/drive/MyDrive/shorts/backgrounds_MP4"  # 可包含 MP4 或 PNG  # 變數指派
CHARACTER_DIR = "/content/drive/MyDrive/shorts/characters"  # 變數指派
OVERLAY_DIR = "/content/drive/MyDrive/shorts/PNG_Overlayers"  # 變數指派
TEXT_STICKER_DIR = "/content/drive/MyDrive/shorts/Text_Overlayers"  # 變數指派
OUTPUT_DIR = "/content/drive/MyDrive/shorts/movie01"  # 變數指派
os.makedirs(OUTPUT_DIR, exist_ok=True)  # 操作路徑或檔案系統

In [7]:
# ===== 主程式流程：對每張貼紙產出影片 =====
text_files = [f for f in os.listdir(TEXT_STICKER_DIR) if f.endswith('.png')]  # 操作路徑或檔案系統
if not text_files:  # if 判斷式
    print("⚠️ 找不到任何貼紙，請確認 TEXT_STICKER_DIR 資料夾內是否有 PNG。")

for text_file in text_files:  # for 迴圈
    # 讀入文字貼紙
    text_path = os.path.join(TEXT_STICKER_DIR, text_file)  # 操作路徑或檔案系統

    # 隨機抽背景
    bg_list = [os.path.join(BACKGROUND_DIR, f) for f in os.listdir(BACKGROUND_DIR) if f.endswith(('.mp4', '.png'))]  # 操作路徑或檔案系統
    if not bg_list:  # if 判斷式
        print("❌ 沒有背景素材，請確認背景資料夾。")
        continue
    bg_path = random.choice(bg_list)  # 隨機數/物件

    # 隨機抽角色與特效
    def rand_img(folder):  # 定義函數：rand_img
        items = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.png')]  # 操作路徑或檔案系統
        return random.choice(items) if items else None  # 回傳結果
    layers = [text_path]  # 最底層為文字貼紙  # 變數指派
    for folder in [CHARACTER_DIR, OVERLAY_DIR]:  # for 迴圈
        picked = rand_img(folder)  # 變數指派
        if picked:  # if 判斷式
            layers.append(picked)

    # 合成影片
    output_path = os.path.join(OUTPUT_DIR, text_file.replace(".png", ".mp4"))  # 操作路徑或檔案系統
    if bg_path.endswith('.mp4'):  # if 判斷式
        overlay_multiple_pngs_on_video_fast(bg_path, layers, output_path)
    else:  # else 其他情況
        # 處理靜態背景圖
        bg_img = cv2.imread(bg_path)  # OpenCV 影像處理函式
        bg_img = cv2.resize(bg_img, (1080, 1920))  # OpenCV 影像處理函式
        overlay_imgs = [Image.open(p).resize((1080, 1920), Image.LANCZOS) for p in layers]  # PIL 影像處理
        result = blend_with_alpha_fast(bg_img, overlay_imgs)  # 變數指派
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 15, (1080, 1920))  # OpenCV 影像處理函式
        for _ in range(75): out.write(result)  # for 迴圈
        out.release()
        print(f"✅ 靜態背景影片產出：{output_path}")

print("🎬 所有影片產出完成！")

✅ 影片已產出：/content/drive/MyDrive/shorts/movie01/text_0.mp4
✅ 影片已產出：/content/drive/MyDrive/shorts/movie01/text_1.mp4
✅ 影片已產出：/content/drive/MyDrive/shorts/movie01/text_2.mp4
🎬 所有影片產出完成！
